In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
# ENABLE IF USING COLAB
USE_COLAB = False

if USE_COLAB:
    import os
    from google.colab import drive
    
    FOLDERNAME = 'UMass_Amherst/NN_AMI/NN_Project'
    drive.mount('/content/drive')
    %cd /content/drive/My\ Drive/$FOLDERNAME

    if os.path.exists('./spectroconv/'):
      %cd ./spectroconv
      !pip install hub
      !pip install hub[audio]
      !pip install wandb
    else:
      print("Error, must first pull project repository from github to Drive using ColabGitCommands.ipynb")

In [3]:
import torch
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: nvidia-smi: command not found


In [82]:
from models import wb_experiment

In [102]:
n_mels = 128
wb_config = {
    'project': 'spectroconv-debug',
    'entity': 'jakeval-colab',
    'group': 'cnn' # use this only when training/evaluating/etc a CNN
}

config = {
    'data': {
        'train': {
            'name': 'nsynth-tiny-train'
        },
        'val': {
            'name': 'nsynth-tiny-val'
        }
    },
    'model': {
        'id': wb_experiment.ModelType.CNN.value,
        'name': 'CNN-tiny'
    },
    'logging': {
        'batch_log_interval': 50,
        'eval_every_epoch': True,
    },
    'parameters': {
        'epochs': 2,
        'batch_size': 64,
        'optimizer': 'adam',
        'learning_rate': 0.0005,
        'dropout_fc': 0.5, # probability of an element to be zeroed (0 = no dropout)
        'dropout_conv': 0.1,
        'dropout_input': 0.2,
        'l2_reg': 0,
        'kernel_size': 3,
        'num_channels': [8, 16],
        'linear_layer_size': 32,
        'max_norm': 0.75,
    },
}

In [103]:
trainer = wb_experiment.TrainExperiment(wb_config=wb_config, login_key=False)
model = trainer.run_train(config, save_model=True)

hub://jakeval/nsynth-tiny-train-metadata loaded successfully.


/home/jakeval/umass/682/project/spectroconv/.env/lib/python3.8/site-packages/hub/core/fast_forwarding.py:43: UserWarning: Loading a dataset that was created or updated with a newer version of hub. This could lead to corruption or unexpected errors! Dataset version: 2.3.5, current hub version: 2.3.2. It's recommended that you update to a version of hub >= 2.3.5.
  warnings.warn(


This dataset can be visualized at https://app.activeloop.ai/jakeval/nsynth-tiny-train-metadata.
hub://jakeval/nsynth-tiny-train loaded successfully.
This dataset can be visualized at https://app.activeloop.ai/jakeval/nsynth-tiny-train.
hub://jakeval/nsynth-tiny-val-metadata loaded successfully.
This dataset can be visualized at https://app.activeloop.ai/jakeval/nsynth-tiny-val-metadata.
hub://jakeval/nsynth-tiny-val loaded successfully.
This dataset can be visualized at https://app.activeloop.ai/jakeval/nsynth-tiny-val.


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Batch 0: Loss: 0.6902, Accuracy: 0.4219


  0%|          | 0/2 [00:00<?, ?it/s]

Batch 0: Loss: 0.4096, Accuracy: 0.6909



train/accuracy,▁█
train/f1,▁█
train/loss,█▁
val/accuracy,▁█
val/f1,▁█
val/loss,█▁
train/accuracy,0.69091
train/f1,0.62939
train/loss,0.40956
val/accuracy,0.86842
val/f1,0.86801


In [104]:
# untested

config = {
    'data': {
        'val': {
            'name': f'nsynth-tiny-val'
        }
    },
    'model_name': 'CNN-tiny',
    'eval': {
        'examples': {
            'k': 8
        }
    }
}


In [105]:
trainer = wb_experiment.EvalExperiment(wb_config)
metrics, examples_df = trainer.run_evaluate(config)

hub://jakeval/nsynth-tiny-val-metadata loaded successfully.


/home/jakeval/umass/682/project/spectroconv/.env/lib/python3.8/site-packages/hub/core/fast_forwarding.py:43: UserWarning: Loading a dataset that was created or updated with a newer version of hub. This could lead to corruption or unexpected errors! Dataset version: 2.3.5, current hub version: 2.3.2. It's recommended that you update to a version of hub >= 2.3.5.
  warnings.warn(


This dataset can be visualized at https://app.activeloop.ai/jakeval/nsynth-tiny-val-metadata.
hub://jakeval/nsynth-tiny-val loaded successfully.
This dataset can be visualized at https://app.activeloop.ai/jakeval/nsynth-tiny-val.
Loading 8 spectrograms...
Loading 8 audio clips...
Opening dataset in read-only mode as you don't have write permissions.
hub://activeloop/nsynth-val loaded successfully.
This dataset can be visualized at https://app.activeloop.ai/activeloop/nsynth-val.
Linished loading



val/accuracy,0.86842
val/f1,0.86801
val/loss,0.42992


## Train on the full dataset

In [ ]:
# Wandb should take care of plotting for us?

'''
from matplotlib import pyplot as plt
fig, ax = plt.subplots()
ax.plot(losses)
fig, ax = plt.subplots()
ax.plot(train_accuracies, label='train acc')
ax.plot(val_accuracies, label='val acc')
ax.plot(val_f1s, label='val f1')
ax.legend()
plt.savefig("learning_metrics.png")
plt.show()
'''

In [ ]:
#print(train_model.stream_accuracy(clf, dl_val, device))

0.7415788789179347


In [ ]:
#print(train_model.stream_f1(clf, dl_val, device))

0.7109029348873568
